In [5]:
import typing as t
from app import crud, models, services, schemas
from app.db.session import SessionLocal
from babel.numbers import format_number
from pydantic import BaseModel
db = SessionLocal()

/usr/local/lib/python3.7/site-packages/jose/backends/cryptography_backend.py:18: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


In [6]:
class FirstPageContext(BaseModel):
    headline: t.Optional[str] = None
    months: t.Optional[t.List[str]] = []
    counts: t.Optional[t.Dict[str, t.List[int]]] = {}
    processing: t.Optional[t.Dict[str, t.List[int]]] = {}
    bills: t.Optional[t.Dict[str, t.Any]] = {}
    benchmarks: t.Optional[t.Dict[str, t.Any]] = {}

In [7]:
class ControllingData(services.ControllingCalendar):
    def __init__(
            self, year: int = None, monthrange: t.List = None,
            store_internal_id: int = None, owner_id: int = None
    ) -> None:
        super().__init__(year=year, monthrange=monthrange)
        self.months_labels = [month for month in self.data.keys() if (month != 'range')]
        self.start = self.data['range']['start']
        self.end = self.data['range']['end']
        self.store_internal_id = store_internal_id
        self.owner_id = owner_id
        self.context: FirstPageContext = FirstPageContext()

    def get_ticket_count_per_kind_and_month(self) -> None:
        for kind in models.TicketKind:
            count_per_month = []
            for m in self.months_labels:
                ticket_count = services.get_tickets_count(
                    db,
                    start=self.data[m]['start'],
                    end=self.data[m]['end'],
                    kind=kind,
                    store_internal_id=self.store_internal_id,
                    owner_id=self.owner_id,
                )
                count_per_month.append(ticket_count)
            self.context.counts[kind.name] = count_per_month
        self.context.counts['TOTAL'] = [
            x + y for (x, y) in zip(
                self.context.counts[models.TicketKind.COMMISSION.name],
                self.context.counts[models.TicketKind.CLAIM.name])
        ]    
        return

    def get_ticket_count_per_status_process(self) -> None:
        for process in models.TicketStatus:
            count_per_process = []
            for m in self.months_labels:
                ticket_count = services.get_tickets_count(
                    db,
                    start=self.data[m]['start'],
                    end=self.data[m]['end'],
                    status=process,
                    store_internal_id=self.store_internal_id,
                    owner_id=self.owner_id,
                )
                count_per_process.append(ticket_count)
            self.context.processing[process.name] = count_per_process
        return

    @staticmethod
    def get_local_format(float_sum: t.Union[float, int], round_int: int = 0) -> str:
        rounded_sum = round(float_sum, round_int)
        local_sum = format_number(rounded_sum, locale='de_DE')
        return local_sum

    def get_clean_claim_bill(self) -> None:
        claim_bill_sum = services.get_claim_bill(
            db,
            start=self.start,
            end=self.end,
            store_internal_id=self.store_internal_id,
            owner_id=self.owner_id,
            calculation="sum"
        )
        claim_discharge_sum = services.get_clean_discharge(
            db,
            start=self.start,
            end=self.end,
            store_internal_id=self.store_internal_id,
            owner_id=self.owner_id,
        )
        clean_bill = claim_discharge_sum + claim_bill_sum
        return clean_bill

    def get_claim_bill_formatted(self) -> float:
        claim_bill_sum = services.get_claim_bill(
            db,
            start=self.start,
            end=self.end,
            store_internal_id=self.store_internal_id,
            owner_id=self.owner_id,
            calculation="sum"
        )
        claim_discharge_sum = services.get_clean_discharge(
            db,
            start=self.start,
            end=self.end,
            store_internal_id=self.store_internal_id,
            owner_id=self.owner_id,
        )
        claim_bill_discharge_dif = claim_bill_sum - claim_discharge_sum
        claim_count = services.get_claim_bill(
            db,
            start=self.start,
            end=self.end,
            store_internal_id=self.store_internal_id,
            owner_id=self.owner_id,
            calculation="count"
        )
        self.context.bills.update({
            'count': claim_count,
            'bill': self.get_local_format(claim_bill_sum),
            'discharge': self.get_local_format(claim_discharge_sum),
            'total': self.get_local_format(claim_bill_discharge_dif),
        })
        return claim_bill_discharge_dif

    def get_benchmarks(self) -> None:
        claim_average = services.get_claim_bill(
            db,
            start=self.start,
            end=self.end,
            store_internal_id=self.store_internal_id,
            owner_id=self.owner_id,
            calculation="avg",
        )
        claim_average_format = self.get_local_format(claim_average, 2)
        ticket_count = sum(self.context.counts[models.TicketKind.COMMISSION.name])
        claim_average_per_commission = self.get_clean_claim_bill() / ticket_count
        claim_average_per_commission_format = self.get_local_format(claim_average_per_commission, 2)
        self.context.benchmarks.update({
            'claim': claim_average_format,
            'commission': claim_average_per_commission_format,
        })
        return

    def get_context(self):
        self.get_ticket_count_per_kind_and_month()
        self.get_ticket_count_per_status_process()
        self.get_claim_bill_formatted()
        self.get_benchmarks()
        return self.context

In [14]:
c = ControllingData(year=2020, owner_id=4)

In [15]:
d = c.get_context()

In [16]:
d.dict()

{'headline': None,
 'months': [],
 'counts': {'IMPORTED': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'COMMISSION': [139, 114, 66, 6, 121, 160, 155, 34, 230, 185, 122, 47],
  'CLAIM': [6, 11, 17, 15, 12, 18, 20, 19, 12, 31, 8, 5],
  'CONNECTED': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'INCOMPLETE': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'TOTAL': [145, 125, 83, 21, 133, 178, 175, 53, 242, 216, 130, 52]},
 'processing': {'NEW': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'OPEN_QUESTION': [1, 0, 0, 0, 0, 0, 0, 0, 1, 6, 23, 15],
  'ANSWERED_QUESTION': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 8, 6],
  'IN_PROGRESS': [5, 0, 0, 0, 1, 1, 2, 0, 4, 9, 17, 16],
  'ORDERED': [2, 1, 2, 0, 3, 13, 7, 8, 66, 135, 73, 14],
  'CLOSED': [137, 124, 81, 21, 129, 164, 166, 45, 170, 65, 9, 1]},
 'bills': {'count': 271,
  'bill': '25.167',
  'discharge': '-6.565',
  'total': '31.732'},
 'benchmarks': {'claim': '92,87', 'commission': '13,49'}}

In [1]:
x = [1, 2, 3]
y = [2, 4, 6]

In [2]:
z = [x + y for (x, y) in zip(x, y)]

In [3]:
z

[3, 6, 9]